# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-1d686c283d-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series (Basketball and NBA seasons explorative search") 

Consider the following exploratory information need:

>  investigate the world of basketball. Compare french teams and US teams, country of citizenship of NBA players, and teams which won NBA seasons.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q41421`| Michael Jordan     | node |
| `wd:Q25369`| Kobe Bryant     | node |





Also consider

```
wd:Q41421 ?p ?obj .
```

is the BGP to retrieve all **properties of Michael Jordan**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for basketball player.

2. Identify the BGP for basketball.

3. Identify the BGP for basketball team.

4. Return all french basketball teams born before 1930 (the result set must be a list of triples basketball team IRI, label and year of inception).

5. Return all the NBA season (the result set must be a list of couples NBA season IRI, label)

6. Consider only the basketball teams which Michael Jordan and Kobe Bryant played. Return the number of NBA seasons won by these teams (the result set must be a list of triples basketball team IRI, label and #victories. Return a number for each basketball team even if they never won a season).

7. Consider only NBA (National Basketball Association) basketball teams and all basketball players who played in one of these teams at least once. Also consider the country of citizenship of the players with the related continent. Return for each continent the number of players who played in an NBA basketball teams. (the result set must be triples of continent IRI, label and #players)

8. Identify the BGP for the NBA Defensive Player of the Year Award (see https://en.wikipedia.org/wiki/NBA_Defensive_Player_of_the_Year_Award)

9. Consider the players who won the NBA Defensive Player of the Year Award. Return for these players the number of NBA Defensive Player of the Year Award won in their career (the result set must be triples of player IRI, label and #victories)


## Task 1
Identify the BGP for basketball player.

In [39]:
# query example
queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
   # bind something
   wd:Q41421 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)



Results
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1048'), ('name', 'NCL ID')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1266'), ('name', 'AlloCiné person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('name', 'CONOR.SI ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('name', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p'

In [7]:
queryString = """
SELECT DISTINCT ?name ?o
WHERE {
   # bind something
   wd:Q41421 ?p ?o .
   # get the label
   ?o sc:name ?name.
}
LIMIT 20
"""
x=run_query(queryString)

[('name', "L'Équipe Champion of Champions"), ('o', 'http://www.wikidata.org/entity/Q1061233')]
[('name', 'entrepreneur'), ('o', 'http://www.wikidata.org/entity/Q131524')]
[('name', 'WikiProject African diaspora'), ('o', 'http://www.wikidata.org/entity/Q15304953')]
[('name', 'Presidential Medal of Freedom'), ('o', 'http://www.wikidata.org/entity/Q17144')]
[('name', 'English'), ('o', 'http://www.wikidata.org/entity/Q1860')]
[('name', 'autobiographer'), ('o', 'http://www.wikidata.org/entity/Q18814623')]
[('name', 'United States of America'), ('o', 'http://www.wikidata.org/entity/Q30')]
[('name', 'right-handedness'), ('o', 'http://www.wikidata.org/entity/Q3039938')]
[('name', 'actor'), ('o', 'http://www.wikidata.org/entity/Q33999')]
[('name', 'basketball player'), ('o', 'http://www.wikidata.org/entity/Q3665646')]
[('name', 'African Americans'), ('o', 'http://www.wikidata.org/entity/Q49085')]
[('name', 'Michael'), ('o', 'http://www.wikidata.org/entity/Q4927524')]
[('name', 'human'), ('o', '

In [13]:
#query the occupation(s) of Michael Jordan
queryString = """
SELECT DISTINCT ?name ?o
WHERE {
   # bind something
   wd:Q41421 wdt:P106 ?o .
   # get the label
   ?o sc:name ?name.
}
LIMIT 20
"""
x=run_query(queryString)

[('name', 'entrepreneur'), ('o', 'http://www.wikidata.org/entity/Q131524')]
[('name', 'autobiographer'), ('o', 'http://www.wikidata.org/entity/Q18814623')]
[('name', 'actor'), ('o', 'http://www.wikidata.org/entity/Q33999')]
[('name', 'basketball player'), ('o', 'http://www.wikidata.org/entity/Q3665646')]
[('name', 'baseball player'), ('o', 'http://www.wikidata.org/entity/Q10871364')]
[('name', 'spokesperson'), ('o', 'http://www.wikidata.org/entity/Q17221')]
6


Final query for this task

In [3]:
# write your final query
#Michael Jordan had more than one occupation, so I asked for the occupation(s) shared by Michael Jordan and Kobe Bryant. Hopefully they only have in common the one I'm interested in
queryString = """
SELECT DISTINCT ?name ?o
WHERE {
   # bind something
   wd:Q25369 wdt:P106 ?o .
   wd:Q41421 wdt:P106 ?o .
   # get the label
   ?o sc:name ?name.
}
LIMIT 20
"""
x=run_query(queryString)

[('name', 'basketball player'), ('o', 'http://www.wikidata.org/entity/Q3665646')]
1


## Task 2
Identify the BGP for basketball.

In [9]:
# write your queries
# looking at what the property "participant" return us for Michael Jordan
queryString = """
SELECT DISTINCT ?name ?o
WHERE {
   # bind something
   wd:Q41421 wdt:P1344 ?o .
   # get the label
   ?o sc:name ?name.
}
LIMIT 20
"""
x=run_query(queryString)

[('name', '1992 Summer Olympics'), ('o', 'http://www.wikidata.org/entity/Q8488')]
[('name', '1992 Tournament of the Americas'), ('o', 'http://www.wikidata.org/entity/Q1985121')]
[('name', '1984 Summer Olympics'), ('o', 'http://www.wikidata.org/entity/Q8456')]
3


In [7]:
queryString = """
SELECT DISTINCT ?name ?o ?namep ?p
WHERE {
   # bind something
   ?player wdt:P31 wd:Q3665646 ;
        ?p ?o .
   # get the label
   ?o sc:name ?name .
   ?p sc:name ?namep .
}
LIMIT 20
"""
x=run_query(queryString)

[('name', 'basketball player'), ('o', 'http://www.wikidata.org/entity/Q3665646'), ('namep', 'occupation'), ('p', 'http://www.wikidata.org/prop/direct/P106')]
[('name', '2009 Mediterranean Games'), ('o', 'http://www.wikidata.org/entity/Q500288'), ('namep', 'participant in'), ('p', 'http://www.wikidata.org/prop/direct/P1344')]
[('name', 'Serbian'), ('o', 'http://www.wikidata.org/entity/Q9299'), ('namep', 'languages spoken, written or signed'), ('p', 'http://www.wikidata.org/prop/direct/P1412')]
[('name', 'Senta'), ('o', 'http://www.wikidata.org/entity/Q571136'), ('namep', 'place of birth'), ('p', 'http://www.wikidata.org/prop/direct/P19')]
[('name', "women's basketball"), ('o', 'http://www.wikidata.org/entity/Q2887217'), ('namep', 'competition class'), ('p', 'http://www.wikidata.org/prop/direct/P2094')]
[('name', 'female'), ('o', 'http://www.wikidata.org/entity/Q6581072'), ('namep', 'sex or gender'), ('p', 'http://www.wikidata.org/prop/direct/P21')]
[('name', 'Serbia'), ('o', 'http://www

Final query for this task

In [9]:
# write your final query
queryString = """
SELECT DISTINCT ?name ?o
WHERE {
   # bind something
   ?player wdt:P31 wd:Q3665646 ;
        wdt:P641 ?o .
   # get the label
   ?o sc:name ?name .
}
LIMIT 20
"""
x=run_query(queryString)

[('name', 'basketball'), ('o', 'http://www.wikidata.org/entity/Q5372')]
1


## Task 3
Identify the BGP for basketball team.

In [10]:
# write your queries
queryString = """
SELECT DISTINCT ?name ?o
WHERE {
   # bind something
   ?player wdt:P31 wd:Q3665646 ;
        wdt:P413 ?o .
   # get the label
   ?o sc:name ?name .
}
LIMIT 20
"""
x=run_query(queryString)

[('name', 'power forward'), ('o', 'http://www.wikidata.org/entity/Q462471')]
1


In [11]:
queryString = """
SELECT DISTINCT ?name ?o ?namep ?p
WHERE {
   # bind something
   ?sport wdt:P31 wd:Q5372 ;
        ?p ?o .
   # get the label
   ?o sc:name ?name .
   ?p sc:name ?namep .
}
LIMIT 20
"""
x=run_query(queryString)

[('name', 'Louisville Gardens'), ('o', 'http://www.wikidata.org/entity/Q6689481'), ('namep', 'home venue'), ('p', 'http://www.wikidata.org/prop/direct/P115')]
[('name', 'SRC Kale'), ('o', 'http://www.wikidata.org/entity/Q7392801'), ('namep', 'home venue'), ('p', 'http://www.wikidata.org/prop/direct/P115')]
[('name', 'Louisville'), ('o', 'http://www.wikidata.org/entity/Q43668'), ('namep', 'headquarters location'), ('p', 'http://www.wikidata.org/prop/direct/P159')]
[('name', 'Skopje'), ('o', 'http://www.wikidata.org/entity/Q384'), ('namep', 'headquarters location'), ('p', 'http://www.wikidata.org/prop/direct/P159')]
[('name', 'United States of America'), ('o', 'http://www.wikidata.org/entity/Q30'), ('namep', 'country'), ('p', 'http://www.wikidata.org/prop/direct/P17')]
[('name', 'North Macedonia'), ('o', 'http://www.wikidata.org/entity/Q221'), ('namep', 'country'), ('p', 'http://www.wikidata.org/prop/direct/P17')]
[('name', 'Kentucky Colonels'), ('o', 'http://www.wikidata.org/entity/Q173

Final query for this task

In [18]:
# write your final query
queryString = """
SELECT DISTINCT ?name ?o
WHERE {
   # bind something
   ?sport wdt:P31 wd:Q5372 ;
        wdt:P31 ?o .
   # get the label
   ?o sc:name ?name .
}
LIMIT 20
"""
x=run_query(queryString)

[('name', 'basketball'), ('o', 'http://www.wikidata.org/entity/Q5372')]
[('name', 'basketball team'), ('o', 'http://www.wikidata.org/entity/Q13393265')]
2


## Task 4
Return all french basketball teams born before 1930 (the result set must be a list of triples basketball team IRI, label and year of inception).

In [8]:
# write your queries
#get all the nationality of the teams, only one result, maybe it's not what we are looking for
queryString = """
SELECT DISTINCT ?nationality ?name
WHERE {
   # bind something
    ?s wdt:P31 wd:Q13393265 ;
     wdt:P27 ?nationality .
    ?nationality sc:name ?name .
}
LIMIT 20
"""
x=run_query(queryString)

[('nationality', 'http://www.wikidata.org/entity/Q148'), ('name', "People's Republic of China")]
1


In [10]:
#we are taking a look to the properties of "basketball team" and we find one that interest us : country.
#we extend the filter to find also the inception property
#we also obtained the 
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   ?s wdt:P31 wd:Q13393265 ;
        ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 60
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P4003'), ('name', 'Facebook page ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1268'), ('name', 'represents')]
[('p', 'http://www.wikidata.org/prop/direct/P4507'), ('name', 'California Sports Hall of Fame athlete ID')]
[('p', 'http://www.wikidata.org/prop/direct/P6518'), ('name', 'Business Registry code (Estonia)')]
[('p', 'http://www.wikidata.org/prop/direct/P9321'), ('name', 'Inforegister ID')]
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('name', 'home venue')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P1320'), ('name', 'OpenCorporates ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direc

In [6]:
#We are now looking for the IRI of France 
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   ?s wdt:P31 wd:Q13393265 ;
        wdt:P17 ?o .
   # get the label
   ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q786'), ('name', 'Dominican Republic')]
[('o', 'http://www.wikidata.org/entity/Q1028'), ('name', 'Morocco')]
[('o', 'http://www.wikidata.org/entity/Q1029'), ('name', 'Mozambique')]
[('o', 'http://www.wikidata.org/entity/Q1037'), ('name', 'Rwanda')]
[('o', 'http://www.wikidata.org/entity/Q1246'), ('name', 'Kosovo')]
[('o', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('o', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom')]
[('o', 'http://www.wikidata.org/entity/Q148'), ('name', "People's Republic of China")]
[('o', 'http://www.wikidata.org/entity/Q155'), ('name', 'Brazil')]
[('o', 'http://www.wikidata.org/entity/Q159'), ('name', 'Russia')]
[('o', 'http://www.wikidata.org/entity/Q16'), ('name', 'Canada')]
[('o', 'http://www.wikidata.org/entity/Q17'), ('name', 'Japan')]
[('o', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('o', 'http://www.wikidata.org/entity/Q184'), ('name', 'Belarus')]
[('o',

Final query for this task

In [4]:
# write your final query
queryString = """
SELECT DISTINCT ?team ?name ?dateOfInception
WHERE {
   ?team wdt:P31 wd:Q13393265 ;        #?team a basketball team
        wdt:P17 wd:Q142;               #from France
        wdt:P571 ?dateOfInception;     
        sc:name ?name .
    FILTER (?dateOfInception < "1930"^^xsd:gYear) .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('team', 'http://www.wikidata.org/entity/Q1682230'), ('name', 'JDA Dijon Basket'), ('dateOfInception', '1880-01-01T00:00:00Z')]
[('team', 'http://www.wikidata.org/entity/Q3496108'), ('name', 'Stade Français'), ('dateOfInception', '1883-01-01T00:00:00Z')]
[('team', 'http://www.wikidata.org/entity/Q3134233'), ('name', 'Hermine de Nantes Atlantique'), ('dateOfInception', '1891-01-01T00:00:00Z')]
[('team', 'http://www.wikidata.org/entity/Q1630255'), ('name', 'SPO Rouen Basket'), ('dateOfInception', '1892-01-01T00:00:00Z')]
[('team', 'http://www.wikidata.org/entity/Q2892731'), ('name', 'Paris Université Club'), ('dateOfInception', '1906-01-01T00:00:00Z')]
[('team', 'http://www.wikidata.org/entity/Q2955800'), ('name', 'Championnet Sports'), ('dateOfInception', '1907-01-01T00:00:00Z')]
[('team', 'http://www.wikidata.org/entity/Q783067'), ('name', 'Alsace de Bagnolet'), ('dateOfInception', '1908-01-01T00:00:00Z')]
[('team', 'http://www.wikidata.org/entity/Q1630650'), ('name', 'JL Bour

## Task 5
Return all the NBA season (the result set must be a list of couples NBA season IRI, label)

In [8]:
# write your queries
#retrieve the leagues of United States basketball teams and we find the IRI of the NBA under the label "National Basketball Association"
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
    ?team wdt:P31 wd:Q13393265 ;
        wdt:P17 wd:Q30 ;
        wdt:P118 ?o .      
    ?o sc:name ?name .    
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)


Results
[('o', 'http://www.wikidata.org/entity/Q204348'), ('name', 'Continental Basketball Association')]
[('o', 'http://www.wikidata.org/entity/Q1125149'), ('name', 'Conference USA')]
[('o', 'http://www.wikidata.org/entity/Q963227'), ('name', 'Southeastern Conference')]
[('o', 'http://www.wikidata.org/entity/Q2593221'), ('name', "Women's National Basketball Association")]
[('o', 'http://www.wikidata.org/entity/Q7312525'), ('name', 'American Athletic Conference')]
[('o', 'http://www.wikidata.org/entity/Q859280'), ('name', 'Big Sky Conference')]
[('o', 'http://www.wikidata.org/entity/Q94861615'), ('name', "NCAA Division I men's basketball")]
[('o', 'http://www.wikidata.org/entity/Q842793'), ('name', 'NBA G League')]
[('o', 'http://www.wikidata.org/entity/Q7820700'), ('name', 'Big East Conference')]
[('o', 'http://www.wikidata.org/entity/Q858430'), ('name', 'Big 12 Conference')]
[('o', 'http://www.wikidata.org/entity/Q729478'), ('name', 'Big Ten Conference')]
[('o', 'http://www.wikidata.

In [6]:
#retrieve all properties from NBA
# we find the properties season ends and season start
queryString = """
SELECT DISTINCT ?p ?name ?o ?nameo
WHERE {
    wd:Q155223 ?p ?o .        
    ?p sc:name ?name .
    ?o sc:name ?nameo .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q18536323'), ('nameo', 'basketball league')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q15986354'), ('nameo', 'basketball association')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('o', 'http://www.wikidata.org/entity/Q238240'), ('nameo', 'Eastern Conference')]
[('p', 'http://www.wikidata.org/prop/direct/P1830'), ('name', 'owner of'), ('o', 'http://www.wikidata.org/entity/Q2455323'), ('nameo', 'NBA TV')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces'), ('o', 'http://www.wikidata.org/entity/Q810343'), ('nameo', 'Basketball Association of America')]
[('p', 'http://www.wikidata.org/prop/direct/P6118'), ('name', 'season ends'), ('o', 'http://www.wikidata.org/entity/Q118'), ('nameo', 'April')]
[('p', 'http://www.wikidata.org/prop/direct/P4794')

In [2]:
#retrieve the subjects of the property "season starts"
#not going anywhere
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P4794 ?o .        
    ?s  sc:name ?name .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q2062651'), ('name', 'Volta de Ciclismo Internacional do Estado de São Paulo')]
[('s', 'http://www.wikidata.org/entity/Q299931'), ('name', 'Chile Open')]
[('s', 'http://www.wikidata.org/entity/Q652498'), ('name', "EHF Women's Champions League")]
[('s', 'http://www.wikidata.org/entity/Q15141004'), ('name', 'Nationalliga A')]
[('s', 'http://www.wikidata.org/entity/Q4288190'), ('name', 'MŽRKL')]
[('s', 'http://www.wikidata.org/entity/Q3009575'), ('name', 'Czech Cycling Tour')]
[('s', 'http://www.wikidata.org/entity/Q1587013'), ('name', 'Ronde van Overijssel')]
[('s', 'http://www.wikidata.org/entity/Q24016079'), ('name', 'Tour of Chongming Island World Cup')]
[('s', 'http://www.wikidata.org/entity/Q685390'), ('name', 'Armenian Supercup')]
[('s', 'http://www.wikidata.org/entity/Q2169635'), ('name', 'Tacx Pro Classic')]
[('s', 'http://www.wikidata.org/entity/Q338901'), ('name', 'Croatian Supercup')]
[('s', 'http://www.wikidata.org/entity/Q508867

In [3]:
#maybe trying to find all the winners?
#nop
queryString = """
SELECT DISTINCT ?team ?name
WHERE {
    wd:Q155223 wdt:P1346 ?team .        
    ?team sc:name ?name .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q121783'), ('name', 'Los Angeles Lakers')]
1


In [13]:
# The Los Angeles Lakers are indicated as the winner of NBA so they must have won the last season so we are trying to find all the objects linked to this team
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
    wd:Q121783 ?p ?o .        
    ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]
[('o', 'http://www.wikidata.org/entity/Q5372'), ('name', 'basketball')]
[('o', 'http://www.wikidata.org/entity/Q65'), ('name', 'Los Angeles')]
[('o', 'http://www.wikidata.org/entity/Q155223'), ('name', 'National Basketball Association')]
[('o', 'http://www.wikidata.org/entity/Q1781055'), ('name', 'South Bay Lakers')]
[('o', 'http://www.wikidata.org/entity/Q186080'), ('name', 'Staples Center')]
[('o', 'http://www.wikidata.org/entity/Q13393265'), ('name', 'basketball team')]
[('o', 'http://www.wikidata.org/entity/Q668549'), ('name', 'Rapperswil-Jona Lakers')]
[('o', 'http://www.wikidata.org/entity/Q206201'), ('name', 'Pacific Division')]
[('o', 'http://www.wikidata.org/entity/Q2456047'), ('name', 'Frank Vogel')]
[('o', 'http://www.wikidata.org/entity/Q4417414'), ('name', 'Category:Los Angeles Lakers')]
[('o', 'http://www.wikidata.org/entity/Q5127792'), ('name', 'Jeanie Buss')]
12


In [8]:
#Nop, maybe doing the other way around
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
    ?s ?p wd:Q121783 ;      
        sc:name ?name .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q17060534'), ('name', '1947–48 Minneapolis Lakers season')]
[('s', 'http://www.wikidata.org/entity/Q538009'), ('name', 'Byron Scott')]
[('s', 'http://www.wikidata.org/entity/Q553063'), ('name', 'Steve Blake')]
[('s', 'http://www.wikidata.org/entity/Q6932895'), ('name', 'Steve Hamilton')]
[('s', 'http://www.wikidata.org/entity/Q2567162'), ('name', 'Jay Vincent')]
[('s', 'http://www.wikidata.org/entity/Q5255286'), ('name', 'Demetrius Calip')]
[('s', 'http://www.wikidata.org/entity/Q2275193'), ('name', 'Mel Counts')]
[('s', 'http://www.wikidata.org/entity/Q2294127'), ('name', '1971–72 NBA season')]
[('s', 'http://www.wikidata.org/entity/Q2552896'), ('name', 'Coby Karl')]
[('s', 'http://www.wikidata.org/entity/Q2562137'), ('name', 'Joe Kleine')]
[('s', 'http://www.wikidata.org/entity/Q2565049'), ('name', 'Zelmo Beaty')]
[('s', 'http://www.wikidata.org/entity/Q2565541'), ('name', 'Elmore Smith')]
[('s', 'http://www.wikidata.org/entity/Q703908')

In [11]:
#there we are, we've finally found the 1971-1972 NBA season.
queryString = """
SELECT DISTINCT ?p ?namep ?o ?name
WHERE {
    wd:Q2294127 ?p ?o .
    ?p sc:name ?namep .
    ?o sc:name ?name .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('namep', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q16772427'), ('name', 'Category:1971–72 NBA season')]
[('p', 'http://www.wikidata.org/prop/direct/P3279'), ('namep', 'statistical leader'), ('o', 'http://www.wikidata.org/entity/Q179051'), ('name', 'Kareem Abdul-Jabbar')]
[('p', 'http://www.wikidata.org/prop/direct/P2348'), ('namep', 'time period'), ('o', 'http://www.wikidata.org/entity/Q53385194'), ('name', '1971-1972 one-year-period')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('namep', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q27020041'), ('name', 'sports season')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('namep', 'country'), ('o', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('namep', 'competition class'), ('o', 'http://www.wikidata.org/entity/Q46343203'), ('name', "men's basketball")

Final query for this task

In [12]:
# write your final query
# we've found there two interesting things. The PGB of organizer and the fact that the season is an instance of sports season.
queryString = """
SELECT DISTINCT ?season ?name 

WHERE {
    ?season wdt:P31 wd:Q27020041 ; #a sport season
        wdt:P664 wd:Q155223 ;      #organized by the NBA         
        sc:name ?name .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('season', 'http://www.wikidata.org/entity/Q600389'), ('name', '1990–91 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q2264531'), ('name', '1967–68 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q241834'), ('name', '1989–90 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q81513156'), ('name', '2019–20 NBA G League season')]
[('season', 'http://www.wikidata.org/entity/Q602553'), ('name', '1969–70 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q2294127'), ('name', '1971–72 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q113619'), ('name', '2012–13 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q16191100'), ('name', '2014-15 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q19862716'), ('name', '2015–16 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q631240'), ('name', '1979–80 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q1142320'), ('name', '2005–06 NBA season')]
[('season', 'http

## Task 6
Consider only the basketball teams which Michael Jordan and Kobe Bryant played. Return the number of NBA seasons won by these teams (the result set must be a list of triples basketball team IRI, label and #victories. Return a number for each basketball team even if they never won a season).

In [2]:
# write your queries
#looking at what property links a basketball player to a basketball team
queryString = """
SELECT DISTINCT ?p ?name

WHERE {  
    wd:Q3665646 ?p wd:Q13393265 .
    ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
1


In [22]:
#we're separating the queries to verify that everything is working well 
#apparently not
queryString = """
SELECT DISTINCT ?team ?name WHERE{
            wd:Q3665646 wdt:P361 ?team .
            ?team wdt:P31 wd:Q13393265 ;
                  sc:name ?name.
            #FILTER (?player = wd:Q41421 || ?player = wd:Q25369) .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [3]:
#the property part of doesn't seem to be the one we are looking for.
queryString = """
SELECT DISTINCT ?team ?name WHERE{
            wd:Q41421 wdt:P361 ?team .
            ?team sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q11310897'), ('name', 'Jordan Rules')]
1


In [27]:
#this time, looking at what property links a basketball team to a basketball player
queryString = """
SELECT DISTINCT ?p ?name

WHERE {  
    wd:Q13393265 ?p wd:Q3665646 .
    ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
1


In [32]:
#still not what we are looking for
queryString = """
SELECT DISTINCT ?team ?name WHERE{
            ?team wdt:P31 wd:Q13393265 ;
                wdt:P527 wd:Q41421 ;
                sc:name ?name.
            #FILTER (?player = wd:Q41421 || ?player = wd:Q25369) .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q369529'), ('name', "1992 United States men's Olympic basketball team")]
1


In [4]:
#we're gonna retrieve what the propriety has part gives us
#Team and squad aren't the same thing ?
queryString = """
SELECT DISTINCT ?o ?name WHERE{
    wd:Q13393265 wdt:P527 ?o .
    ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q51747567'), ('name', 'squad')]
[('o', 'http://www.wikidata.org/entity/Q3665646'), ('name', 'basketball player')]
[('o', 'http://www.wikidata.org/entity/Q5137571'), ('name', 'basketball coach')]
3


In [38]:
#we're gonna look at what there is in the squad object
queryString = """
SELECT DISTINCT ?p ?namep ?o ?name WHERE{
    wd:Q51747567 ?p ?o .
    ?p sc:name ?namep .
    ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('namep', 'has part'), ('o', 'http://www.wikidata.org/entity/Q18536342'), ('name', 'competitive player')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('namep', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q16334295'), ('name', 'group of humans')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('namep', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q327245'), ('name', 'team')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('namep', 'part of'), ('o', 'http://www.wikidata.org/entity/Q847017'), ('name', 'sports club')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('namep', 'has part'), ('o', 'http://www.wikidata.org/entity/Q2066131'), ('name', 'athlete')]
5


In [5]:
#not going anywhere
queryString = """
SELECT DISTINCT ?club ?name WHERE{
    ?squad wdt:P31 wd:Q51747567 ;
            wdt:P361 ?club .
    ?club sc:name ?name .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('club', 'http://www.wikidata.org/entity/Q8558'), ('name', '2004 Summer Olympics')]
[('club', 'http://www.wikidata.org/entity/Q8567'), ('name', '2008 Summer Olympics')]
[('club', 'http://www.wikidata.org/entity/Q102720'), ('name', 'Hertha BSC')]
[('club', 'http://www.wikidata.org/entity/Q81274'), ('name', 'FIBA EuroBasket 2009')]
[('club', 'http://www.wikidata.org/entity/Q1316347'), ('name', '2017 Africa Cup of Nations')]
[('club', 'http://www.wikidata.org/entity/Q740936'), ('name', "handball at the 2004 Summer Olympics – men's tournament")]
[('club', 'http://www.wikidata.org/entity/Q24091451'), ('name', "2018 AFC Women's Asian Cup")]
[('club', 'http://www.wikidata.org/entity/Q18851388'), ('name', "2015 King's Cup")]
8


In [8]:
#maybe we can restart from the Lakers and one of the people who were indicated as subject of the properties, like Byron Scott to see what relation there is between them
#yes finally something interesting.
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
    wd:Q538009 ?p wd:Q121783 .     
    ?p sc:name ?name .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P54'), ('name', 'member of sports team')]
1


In [10]:
#verifying separately again
queryString = """
SELECT DISTINCT ?team ?name WHERE{
            ?player wdt:P106 wd:Q3665646 ;
                wdt:P54 ?team .
            ?team wdt:P31 wd:Q13393265 ;
                  sc:name ?name.
            FILTER (?player = wd:Q41421 || ?player = wd:Q25369) .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q121783'), ('name', 'Los Angeles Lakers')]
[('team', 'http://www.wikidata.org/entity/Q128109'), ('name', 'Chicago Bulls')]
[('team', 'http://www.wikidata.org/entity/Q169165'), ('name', 'Washington Wizards')]
[('team', 'http://www.wikidata.org/entity/Q4412061'), ('name', "North Carolina Tar Heels men's basketball")]
4


In [20]:
queryString = """
SELECT DISTINCT ?name (COUNT(?team) as ?victories)
WHERE {
    ?team wdt:P31 wd:Q13393265 ;
        sc:name ?name .    
    ?season wdt:P31 wd:Q27020041 ; #a sport season
        wdt:P664 wd:Q155223 ;      #organized by the NBA         
        wdt:P1346 ?team. 
} GROUP BY ?name
ORDER BY DESC(?victories)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('name', 'Los Angeles Lakers'), ('victories', '17')]
[('name', 'Boston Celtics'), ('victories', '17')]
[('name', 'Chicago Bulls'), ('victories', '6')]
[('name', 'Golden State Warriors'), ('victories', '6')]
[('name', 'San Antonio Spurs'), ('victories', '5')]
[('name', 'Miami Heat'), ('victories', '3')]
[('name', 'Philadelphia 76ers'), ('victories', '3')]
[('name', 'Milwaukee Bucks'), ('victories', '2')]
[('name', 'New York Knicks'), ('victories', '2')]
[('name', 'Houston Rockets'), ('victories', '2')]
[('name', 'Detroit Pistons'), ('victories', '2')]
[('name', 'Portland Trail Blazers'), ('victories', '1')]
[('name', 'Sacramento Kings'), ('victories', '1')]
[('name', 'Atlanta Hawks'), ('victories', '1')]
[('name', 'Cleveland Cavaliers'), ('victories', '1')]
[('name', 'Washington Wizards'), ('victories', '1')]
[('name', 'Lakeland Magic'), ('victories', '1')]
[('name', 'Baltimore Bullets'), ('victories', '1')]
[('name', 'Toronto Raptors'), ('victories', '1')]
[('name', 'Dallas Ma

In [30]:
#we see that one team is missing, surely cause it didn't win
queryString = """
SELECT DISTINCT ?team2 ?name2 (COUNT(?team2) as ?victories)

WHERE {
    ?team2 wdt:P31 wd:Q13393265 ;
        sc:name ?name2 .
    ?season wdt:P31 wd:Q27020041 ; #a sport season
    wdt:P664 wd:Q155223 ;      #organized by the NBA         
    wdt:P1346 ?team2.


    {
        SELECT DISTINCT ?team ?name WHERE{
            ?player wdt:P106 wd:Q3665646 ;
                wdt:P54 ?team .
            ?team wdt:P31 wd:Q13393265 ;
                  sc:name ?name.
            FILTER (?player = wd:Q41421 || ?player = wd:Q25369)
        }
    } FILTER(?team2 = ?team)
} GROUP BY ?team2 ?name2
ORDER BY DESC(?victories)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('team2', 'http://www.wikidata.org/entity/Q121783'), ('name2', 'Los Angeles Lakers'), ('victories', '17')]
[('team2', 'http://www.wikidata.org/entity/Q128109'), ('name2', 'Chicago Bulls'), ('victories', '6')]
[('team2', 'http://www.wikidata.org/entity/Q169165'), ('name2', 'Washington Wizards'), ('victories', '1')]
3


In [32]:
#We're then adding optional to see the last one. Problem, as it has created one row for the Tar Heels the COUNT take it in account and put 1 as the number of victories
queryString = """
SELECT DISTINCT ?team2 ?name2 (COUNT(?team2) as ?victories)

WHERE {
    ?team2 wdt:P31 wd:Q13393265 ;
        sc:name ?name2 .
    OPTIONAL{
        ?season wdt:P31 wd:Q27020041 ; #a sport season
        wdt:P664 wd:Q155223 ;      #organized by the NBA         
        wdt:P1346 ?team2.
        } 

    {
        SELECT DISTINCT ?team ?name WHERE{
            ?player wdt:P106 wd:Q3665646 ;
                wdt:P54 ?team .
            ?team wdt:P31 wd:Q13393265 ;
                  sc:name ?name.
            FILTER (?player = wd:Q41421 || ?player = wd:Q25369)
        }
    } FILTER(?team2 = ?team)
} GROUP BY ?team2 ?name2
ORDER BY DESC(?victories)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('team2', 'http://www.wikidata.org/entity/Q121783'), ('name2', 'Los Angeles Lakers'), ('victories', '17')]
[('team2', 'http://www.wikidata.org/entity/Q128109'), ('name2', 'Chicago Bulls'), ('victories', '6')]
[('team2', 'http://www.wikidata.org/entity/Q169165'), ('name2', 'Washington Wizards'), ('victories', '1')]
[('team2', 'http://www.wikidata.org/entity/Q4412061'), ('name2', "North Carolina Tar Heels men's basketball"), ('victories', '1')]
4


Final query for this task

In [33]:
# write your final query
#problem finally solved, we'll remember this query all my life

queryString = """
SELECT DISTINCT ?team2 ?name2 (SUM (IF(?team2 = ?winner, 1, 0)) as ?victories) 

WHERE {
    ?team2 wdt:P31 wd:Q13393265 ;
        sc:name ?name2 .

    ?season wdt:P31 wd:Q27020041 ; #a sport season
        wdt:P664 wd:Q155223 ;      #organized by the NBA         
        wdt:P1346 ?winner.
        
    {
        SELECT DISTINCT ?team ?name WHERE{
            ?player wdt:P106 wd:Q3665646 ;
                wdt:P54 ?team .
            ?team wdt:P31 wd:Q13393265 ;
                  sc:name ?name.
            FILTER (?player = wd:Q41421 || ?player = wd:Q25369)
        }
    } FILTER(?team2 = ?team)
} GROUP BY ?team2 ?name2 
ORDER BY DESC(?victories)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('team2', 'http://www.wikidata.org/entity/Q121783'), ('name2', 'Los Angeles Lakers'), ('victories', '17')]
[('team2', 'http://www.wikidata.org/entity/Q128109'), ('name2', 'Chicago Bulls'), ('victories', '6')]
[('team2', 'http://www.wikidata.org/entity/Q169165'), ('name2', 'Washington Wizards'), ('victories', '1')]
[('team2', 'http://www.wikidata.org/entity/Q4412061'), ('name2', "North Carolina Tar Heels men's basketball"), ('victories', '0')]
4


## Task 7
Consider only NBA (National Basketball Association) basketball teams and all basketball players who played in one of these teams at least once. Also consider the country of citizenship of the players with the related continent. Return for each continent the number of players who played in an NBA basketball teams. (the result set must be triples of continent IRI, label and #players)

In [31]:
# write your queries
#we don't seem to know the property linking a country to a continent. So we're trying to obtain it retrieving the property of France
queryString = """
SELECT DISTINCT ?p ?name

WHERE {
    wd:Q142 ?p ?o.
    ?p sc:name ?name.
    FILTER(!isLiteral(?o))
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)



Results
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('name', 'basic form of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1313'), ('name', 'office held by head of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1325'), ('name', 'external data available at')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1464'), ('name', 'category for people born here')]
[('p', 'http://www.wikidata.org/prop/direct/P1465'), ('name', 'category for people who died here')]
[('p', 'http://www.wikidata.org/prop/direct/P1482'), ('name', 'Stack Exchange tag')]
[('p', 'http:

In [27]:
#prequerie to verify step by step
queryString = """
SELECT DISTINCT ?country ?name (COUNT(?player) as ?nbplayers) WHERE {
    ?player wdt:P106 wd:Q3665646 ; 
            wdt:P54 ?team ;
            wdt:P27 ?country . 
            
   ?team wdt:P31 wd:Q13393265 ; #a basketball team
        wdt:P118 wd:Q155223 .            #has league NBA
    ?country sc:name ?name.
} GROUP BY ?country ?name
ORDER BY DESC(?nbplayers)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America'), ('nbplayers', '9302')]
[('country', 'http://www.wikidata.org/entity/Q16'), ('name', 'Canada'), ('nbplayers', '86')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France'), ('nbplayers', '68')]
[('country', 'http://www.wikidata.org/entity/Q403'), ('name', 'Serbia'), ('nbplayers', '59')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('name', 'Spain'), ('nbplayers', '51')]
[('country', 'http://www.wikidata.org/entity/Q224'), ('name', 'Croatia'), ('nbplayers', '50')]
[('country', 'http://www.wikidata.org/entity/Q1033'), ('name', 'Nigeria'), ('nbplayers', '49')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy'), ('nbplayers', '46')]
[('country', 'http://www.wikidata.org/entity/Q408'), ('name', 'Australia'), ('nbplayers', '46')]
[('country', 'http://www.wikidata.org/entity/Q43'), ('name', 'Turkey'), ('nbplayers', '45')]
[('country', 'http://www.wikidata

Final query for this task

In [28]:
# write your final query
#doing it with the continent instead of the countries, still without the team thing.
queryString = """
SELECT DISTINCT ?continent ?name (COUNT(?player) as ?nbplayers) WHERE {
    ?player wdt:P106 wd:Q3665646 ; 
            wdt:P54 ?team ;
            wdt:P27 ?country .   
    ?team wdt:P31 wd:Q13393265 ; #a basketball team
        wdt:P118 wd:Q155223 .            #has league NBA
    ?country wdt:P30 ?continent.
    ?continent sc:name ?name.
} GROUP BY ?continent ?name
ORDER BY DESC(?nbplayers)
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('continent', 'http://www.wikidata.org/entity/Q49'), ('name', 'North America'), ('nbplayers', '9521')]
[('continent', 'http://www.wikidata.org/entity/Q538'), ('name', 'Insular Oceania'), ('nbplayers', '9352')]
[('continent', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe'), ('nbplayers', '792')]
[('continent', 'http://www.wikidata.org/entity/Q48'), ('name', 'Asia'), ('nbplayers', '178')]
[('continent', 'http://www.wikidata.org/entity/Q15'), ('name', 'Africa'), ('nbplayers', '131')]
[('continent', 'http://www.wikidata.org/entity/Q18'), ('name', 'South America'), ('nbplayers', '80')]
[('continent', 'http://www.wikidata.org/entity/Q5401'), ('name', 'Eurasia'), ('nbplayers', '45')]
7


## Task 8
Identify the BGP for the NBA Defensive Player of the Year Award (see https://en.wikipedia.org/wiki/NBA_Defensive_Player_of_the_Year_Award)

In [34]:
# write your queries
# we found P166 "award received" when looking at the property of France we're gonna use that
queryString = """
SELECT DISTINCT ?o ?name

WHERE {
    ?player wdt:P106 wd:Q3665646 ;
            wdt:P166 ?o .
    ?o sc:name ?name .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)


Results
[('o', 'http://www.wikidata.org/entity/Q671622'), ('name', 'Arjuna Award')]
[('o', 'http://www.wikidata.org/entity/Q5044310'), ('name', 'Carol Eckman Award')]
[('o', 'http://www.wikidata.org/entity/Q913910'), ('name', 'Order of the Three Stars')]
[('o', 'http://www.wikidata.org/entity/Q7569437'), ('name', "Southeastern Conference Men's Basketball Player of the Year")]
[('o', 'http://www.wikidata.org/entity/Q16998141'), ('name', 'Kay Yow Award')]
[('o', 'http://www.wikidata.org/entity/Q3304648'), ('name', "Atlantic Coast Conference Men's Basketball Player of the Year")]
[('o', 'http://www.wikidata.org/entity/Q586458'), ('name', 'FIBA Hall of Fame')]
[('o', 'http://www.wikidata.org/entity/Q644357'), ('name', 'NBA Rookie of the Year Award')]
[('o', 'http://www.wikidata.org/entity/Q3869345'), ('name', 'NABC Defensive Player of the Year')]
[('o', 'http://www.wikidata.org/entity/Q24915026'), ('name', 'WBCA Defensive Player of the Year')]
[('o', 'http://www.wikidata.org/entity/Q181566

Final query for this task

In [3]:
# write your final query
queryString = """
SELECT DISTINCT ?o ?name

WHERE {
    ?player wdt:P106 wd:Q3665646 ;
            wdt:P166 ?o .
    ?o sc:name ?name .
    FILTER REGEX(?name, "^NBA.*")
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('o', 'http://www.wikidata.org/entity/Q644357'), ('name', 'NBA Rookie of the Year Award')]
[('o', 'http://www.wikidata.org/entity/Q18156668'), ('name', 'NBA Championship ring')]
[('o', 'http://www.wikidata.org/entity/Q1465181'), ('name', 'NBA All-Defensive Team')]
[('o', 'http://www.wikidata.org/entity/Q845884'), ('name', 'NBA Defensive Player of the Year Award')]
[('o', 'http://www.wikidata.org/entity/Q918054'), ('name', 'NBA Coach of the Year Award')]
[('o', 'http://www.wikidata.org/entity/Q1418009'), ('name', 'NBA All-Rookie Team')]
[('o', 'http://www.wikidata.org/entity/Q1418596'), ('name', 'NBA Sportsmanship Award')]
[('o', 'http://www.wikidata.org/entity/Q388970'), ('name', 'NBA Executive of the Year Award')]
[('o', 'http://www.wikidata.org/entity/Q222047'), ('name', 'NBA Most Valuable Player Award')]
[('o', 'http://www.wikidata.org/entity/Q1125399'), ('name', 'NBA Most Improved Player Award')]
[('o', 'http://www.wikidata.org/entity/Q137341'), ('name', 'NBA All-Star Game

## Task 9
Consider the players who won the NBA Defensive Player of the Year Award. Return for these players the number of NBA Defensive Player of the Year Award won in their career (the result set must be triples of player IRI, label and #victories)

In [13]:
# write your queries
# it seems that there is noone who win this award more than one time in the data given(which is strange since according to wikipedia, some people won it multiple times)
queryString = """
SELECT DISTINCT ?player ?name

WHERE {
    ?player wdt:P106 wd:Q3665646 ;
            wdt:P166 wd:Q845884 ;
            sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q177928'), ('name', 'Sidney Moncrief')]
[('player', 'http://www.wikidata.org/entity/Q310968'), ('name', 'Alonzo Mourning')]
[('player', 'http://www.wikidata.org/entity/Q971538'), ('name', 'Michael Cooper')]
[('player', 'http://www.wikidata.org/entity/Q189240'), ('name', 'Kevin Garnett')]
[('player', 'http://www.wikidata.org/entity/Q41421'), ('name', 'Michael Jordan')]
[('player', 'http://www.wikidata.org/entity/Q273256'), ('name', 'Hakeem Olajuwon')]
[('player', 'http://www.wikidata.org/entity/Q211876'), ('name', 'David Robinson')]
[('player', 'http://www.wikidata.org/entity/Q361134'), ('name', 'Marcus Camby')]
[('player', 'http://www.wikidata.org/entity/Q295322'), ('name', 'Metta Sandiford-Artest')]
[('player', 'http://www.wikidata.org/entity/Q926248'), ('name', 'Kawhi Leonard')]
[('player', 'http://www.wikidata.org/entity/Q311735'), ('name', 'Joakim Noah')]
[('player', 'http://www.wikidata.org/entity/Q449245'), ('name', 'Alvin Rober

In [12]:
queryString = """
SELECT DISTINCT ?p ?name ?o ?nameo

WHERE {
    wd:Q845884 ?p ?o.
    ?p sc:name ?name.
    ?o sc:name ?nameo.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner'), ('o', 'http://www.wikidata.org/entity/Q177928'), ('nameo', 'Sidney Moncrief')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q618779'), ('nameo', 'award')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner'), ('o', 'http://www.wikidata.org/entity/Q310968'), ('nameo', 'Alonzo Mourning')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('o', 'http://www.wikidata.org/entity/Q5372'), ('nameo', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner'), ('o', 'http://www.wikidata.org/entity/Q971538'), ('nameo', 'Michael Cooper')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner'), ('o', 'http://www.wikidata.org/entity/Q189240'), ('nameo', 'Kevin Garnett')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner'), ('o', 'http://www.wikidata

Final query for this task

In [9]:
# write your final query
queryString = """
SELECT DISTINCT ?player ?name (COUNT(?player) as ?victory)

WHERE {
    ?player wdt:P106 wd:Q3665646 ;
            wdt:P166 wd:Q845884 ;
            sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q41421'), ('name', 'Michael Jordan'), ('victory', '1')]
[('player', 'http://www.wikidata.org/entity/Q311735'), ('name', 'Joakim Noah'), ('victory', '1')]
[('player', 'http://www.wikidata.org/entity/Q926248'), ('name', 'Kawhi Leonard'), ('victory', '1')]
[('player', 'http://www.wikidata.org/entity/Q8991894'), ('name', 'Giannis Antetokounmpo'), ('victory', '1')]
[('player', 'http://www.wikidata.org/entity/Q25278'), ('name', 'Dwight Howard'), ('victory', '1')]
[('player', 'http://www.wikidata.org/entity/Q295364'), ('name', 'Ben Wallace'), ('victory', '1')]
[('player', 'http://www.wikidata.org/entity/Q971538'), ('name', 'Michael Cooper'), ('victory', '1')]
[('player', 'http://www.wikidata.org/entity/Q3446792'), ('name', 'Rudy Gobert'), ('victory', '1')]
[('player', 'http://www.wikidata.org/entity/Q310968'), ('name', 'Alonzo Mourning'), ('victory', '1')]
[('player', 'http://www.wikidata.org/entity/Q361134'), ('name', 'Marcus Camby'), ('vic